## セクション5:動画処理・動画解析

#### 40.色検出

In [1]:
import cv2
import numpy as np

In [6]:
cap = cv2.VideoCapture('data/movie/Mobility.mp4')
while True:
    cv2.namedWindow('img', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('img', 640, 480)
    ret, frame = cap.read()
    if ret == False:
        break
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower = np.array([20, 50, 50]) #黄色定義の下限
    upper = np.array([25, 255, 255]) #黄色定義の上限
    frame_mask = cv2.inRange(hsv, lower, upper)
    dst = cv2.bitwise_and(frame, frame, mask = frame_mask) #2値画像の論理積で共通する部分を求める
    cv2.imshow('img', dst)
    if cv2.waitKey(10) == 27: #Esc押すと終了
        break
cv2.destroyAllWindows()

#### 41.オプティカルフロー

In [31]:
cv2.namedWindow('img', cv2.WINDOW_NORMAL)
cv2.resizeWindow('img', 1200, 800)
COUNT = 500  # 特徴点の設定
criteria = (cv2.TERM_CRITERIA_MAX_ITER | cv2.TERM_CRITERIA_EPS, 20, 0.03) # 収束条件
lk_params = dict(winSize=(10,10), maxLevel=4, criteria=criteria)
cap = cv2.VideoCapture('data/movie/Cosmos.mp4')
ret, frame = cap.read()
frame_pre = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
while True:
    ret, frame = cap.read()
    if ret == False:
        break
    frame_now = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    feature_pre = cv2.goodFeaturesToTrack(frame_pre, COUNT, 0.001, 5)
    if feature_pre is None:
        continue
    feature_now, status, err = cv2.calcOpticalFlowPyrLK(frame_pre, frame_now, feature_pre, None, **lk_params)
    for i in range(COUNT):
        pre_x = feature_pre[i][0][0]
        pre_y = feature_pre[i][0][1]
        now_x = feature_now[i][0][0]
        now_y = feature_now[i][0][1]
        cv2.line(frame, (pre_x, pre_y), (now_x, now_y), (255, 0, 0), 3)
    cv2.imshow('img', frame)
    frame_pre = frame_now.copy()
    if cv2.waitKey(10) == 27:
        break        
cv2.destroyAllWindows()

#### 42. MeanShift・CamShift

In [39]:
cap = cv2.VideoCapture('data/movie/Cruse.mp4')
ret, frame = cap.read()
h, w, ch = frame.shape

In [34]:
# meanShiftの場合
rct = (600, 500, 200, 200) #検索窓の開始点座標と窓の大きさ
cv2.namedWindow('win', cv2.WINDOW_NORMAL)
cv2.resizeWindow('win', 1200, 800)
cri = (cv2.TERM_CRITERIA_COUNT | cv2.TERM_CRITERIA_EPS, 10, 1) #収束条件
while True:
    threshold = 100
    ret, frame = cap.read()
    if ret == False:
        break
    img_g  = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    ret, img_bin = cv2.threshold(img_g, threshold, 255, cv2.THRESH_BINARY) #2値画像を取得
    ret, rct = cv2.meanShift(img_bin, rct, cri)
    x, y, w, h = rct
    frame = cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0),3)
    cv2.imshow('win', frame)
    if cv2.waitKey(10) == 27:
        break
cv2.destroyAllWindows()

In [40]:
# CamShiftの場合
rct = (600, 500, 200, 200) #検索窓の開始点座標と窓の大きさ
cv2.namedWindow('win', cv2.WINDOW_NORMAL)
cv2.resizeWindow('win', 1200, 800)
cri = (cv2.TERM_CRITERIA_COUNT | cv2.TERM_CRITERIA_EPS, 10, 1) #収束条件
while True:
    threshold = 100
    ret, frame = cap.read()
    if ret == False:
        break
    img_g  = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    ret, img_bin = cv2.threshold(img_g, threshold, 255, cv2.THRESH_BINARY) #2値画像を取得
    ret, rct = cv2.CamShift(img_bin, rct, cri)
    x, y, w, h = rct
    frame = cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0),3)
    cv2.imshow('win', frame)
    if cv2.waitKey(10) == 27:
        break
cv2.destroyAllWindows()

#### 43.背景差分

In [45]:
cv2.namedWindow('img', cv2.WINDOW_NORMAL)
cv2.resizeWindow('img', 1200, 800)
cap = cv2.VideoCapture('data/movie/Pepole.mp4')
ret, frame = cap.read()
h, w, ch = frame.shape
frame_back = np.zeros((h, w, ch), dtype=np.float32)
while True:
    ret, frame = cap.read()
    if ret == False:
        break
    frame_diff = cv2.absdiff(frame.astype(np.float32), frame_back)
    cv2.accumulateWeighted(frame, frame_back, 0.03) #frame_backに3%づつ元画像を混ぜる
    cv2.imshow('img', frame_diff.astype(np.uint8))
    if cv2.waitKey(10) == 27:
        break
cv2.destroyAllWindows()

#### 45.パーティクルフィルター

In [46]:
import cv2
import numpy as np
import random2
import likelihood as li

In [50]:
cap = cv2.VideoCapture('data/movie/Tram.mp4')
ret, frame = cap.read()
h, w = frame.shape[:2]
np.random.seed(100)
Np = 500 # 粒子の設定
px = np.zeros((Np), dtype=np.int64)
py = np.zeros((Np), dtype=np.int64)
lp = np.zeros((Np)) #粒子の尤度を入れる箱
for i in range(Np):
    px[i] = int(np.random.uniform(0,w))
    py[i] = int(np.random.uniform(0,h))
obj = [0, 110, 160] #動画の黄色い部分を指定
while True:
    ret, frame = cap.read()
    if ret == False:
        break
    lp = li.likelihood(frame, px, py, obj, Np, sigma2=0.001)
    pxnew = np.array(random2.choices(population=px, weights=lp, k = Np)) + np.random.randint(-15, 15, Np)
    pynew = np.array(random2.choices(population=py, weights=lp, k = Np)) + np.random.randint(-15, 15, Np)
    px = np.where(pxnew > w -1, w -1, pxnew) #画面からはみ出した時の処理
    py = np.where(pynew > h -1, h -1, pynew) #画面からはみ出した時の処理
    px = np.where(px < 0, 0, px)
    py = np.where(py < 0, 0, py)
    for i in range(Np):
        cv2.circle(frame, (px[i],py[i]), 1, (0, 255, 0), 1)
    cv2.imshow('img', frame)
    if cv2.waitKey(10) == 27:
        break
cv2.destroyAllWindows()

In [52]:
import os
os.listdir()

['.ipynb_checkpoints',
 'data',
 'likelihood.py',
 'output',
 'random2.py',
 'section3.ipynb',
 'section4.ipynb',
 'section5.ipynb',
 '__pycache__']